# Анализ мошенничества с кредитными картами
<br>**Описание задачи**
<br>Данные представляют собой транзакции, совершенные с помощью кредитных карт в сентябре
2013 года европейскими держателями карт. <br>В датасете содержится информация о 284,807
транзакциях, включая 492 случая мошенничества, что составляет 0.172% от общего числа
транзакций. <br>Данные представлены числовыми переменными, полученными в результате
преобразования методом главных компонент (PCA). <br>Оригинальные признаки и подробности о
данных не предоставлены из соображений конфиденциальности. <br>Признаки V1-V28 являются
результатом PCA, <br>'Time' содержит время в секундах между транзакцией и первой записью,
<br>а 'Amount' обозначает сумму транзакции.

**Ход решения**

1. Корреляционный анализ:      - Рассчитать корреляционную матрицу для числовых переменных
    - - Возвращаемый тип данных - numpy массив с размерностью 2.
2. Проверка статистических гипотез    
  - Проверить гипотезы, например, о равенстве средних сумм для признако     
  - Параметры включают признаки для сравнения, критерий проверки (t-test, z-test, kolmogorov-smirnov, fisher) и уровень значимости.
3. Точечные оценки параметров распределе    я: 
  - Оценить средние и дисперсии для выбранных признаков с использованием метода моментов и метода максимального правдопод    ия. 
  - Возвращаемый тип данных - строка с оценками.
4. Определение влияния признаков на безопасность тран    кции: 
  - Применить логистическую регрессию для выявления признаков, оказывающих наибольшее влияние на безопасность тра    акций. 
  - Возвращаемый тип данных - список с названиями признаков на основе весов в логистической регрессии, ограниченный top_k параметрами.
5. Интервальн     оценки: 
  - Рассчитать доверительные интервалы для среднего значения выбранного признака с учетом уровня     ачимости. 
  - Использовать t- или z-статистику в зависимости от размера выборки (подсказка: центральная предельная теорема и з    чение 30). 
  - Возвращаемый тип данных - tuple с интервальной оценкой в формате (lower_bound, upper_bound).

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

## Знакомство с данными

In [4]:
df = pd.read_csv('creditcard.csv')
df.info()
df.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
61674,49931.0,-0.626867,1.153389,1.023489,-0.230298,0.301505,-0.014857,0.498235,0.275810,-0.351408,...,-0.244658,-0.596432,-0.019307,-0.380542,-0.189787,0.111337,0.363092,0.137999,8.99,0
29163,35347.0,-2.557198,-2.173081,2.703097,0.932893,1.223349,-1.353648,-0.549955,0.176163,0.554154,...,0.387662,0.312358,0.416233,0.314071,0.598483,-0.354208,-0.231268,-0.092619,180.00,0
169277,119612.0,-0.445856,1.480038,-0.829392,-0.658226,0.867111,-0.727818,0.868708,0.024381,0.058156,...,-0.369557,-0.805487,0.066451,0.393816,-0.327703,0.087634,-0.078829,-0.156503,9.12,0


Соотношение меток в целевом признаке

In [7]:
print(df['Class'].value_counts())
df['Class'].value_counts()[1] / df['Class'].value_counts()[0]

Class
0    284315
1       492
Name: count, dtype: int64


0.0017304750013189597

## EDA